In [ ]:
# default_exp training_module

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from logging import raiseExceptions
from typing import Tuple
from fastcore.foundation import L
from counterfactual.import_essentials import *
from counterfactual.utils import *
from torchmetrics.functional import accuracy
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OneHotEncoder
import higher
from torch.optim.lr_scheduler import CyclicLR
from counterfactual.interface import ABCBaseModule, GlobalExplainerBase

In [ ]:
print(f"pl version: {pl.__version__}")
print(f"torch version: {torch.__version__}")

pl version: 1.1.0
torch version: 1.7.1


In [ ]:
%%time
dummy_data = pd.read_csv('../data/dummy_data.csv')
adult_data = load_adult_income_dataset('../data/adult.data')

CPU times: user 1.02 s, sys: 284 ms, total: 1.31 s
Wall time: 1.3 s


## Base Class

In [ ]:
# hide
scalar = StandardScaler()
cont = scalar.fit_transform(adult_data[['age', 'hours_per_week']])
scalar.inverse_transform(cont)

array([[39., 40.],
       [50., 13.],
       [38., 40.],
       ...,
       [58., 40.],
       [22., 20.],
       [52., 40.]])

In [ ]:
enc = OneHotEncoder()
cat  = enc.fit_transform(adult_data[['workclass','education', 'marital_status', 
            'occupation','race', 'gender']])
enc.inverse_transform(cat.todense())

array([['Government', 'Bachelors', 'Single', 'White-Collar', 'White',
        'Male'],
       ['Self-Employed', 'Bachelors', 'Married', 'White-Collar', 'White',
        'Male'],
       ['Private', 'HS-grad', 'Divorced', 'Blue-Collar', 'White', 'Male'],
       ...,
       ['Private', 'HS-grad', 'Widowed', 'White-Collar', 'White',
        'Female'],
       ['Private', 'HS-grad', 'Single', 'White-Collar', 'White', 'Male'],
       ['Self-Employed', 'HS-grad', 'Married', 'White-Collar', 'White',
        'Female']], dtype=object)

In [ ]:
# export
class BaseModule(pl.LightningModule, ABCBaseModule):
    _DEFAULT_PARAMS = {
        'encoder': None,
        'normalizer': None,
        'enc_dims': [],
        'dec_dims': [],
        'exp_dims': [],
        'lr': 3e-3,
        'batch_size': 128,
        'sample_frac': None,
        'lambda_1': 1.0,
        'lambda_2': 1.0,
        'lambda_3': 1.0,
        'threshold': 0.5,
        'smooth_y': True
    }

    def __init__(self, config: Dict):
        super().__init__()
        self.save_hyperparameters(config)

        # read data
        self.data = pd.read_csv(Path(config['data_dir']))

        # update default parameters
        self._update_params(config)

        # update cat_idx
        self.cat_idx = len(self.continous_cols)

        # log graph
        self.example_input_array = torch.randn((1, self.enc_dims[0]))


    def _update_params(self, config: Dict[str, Any]):
        self._DEFAULT_PARAMS.update(config)
        for k, v in self._DEFAULT_PARAMS.items():
            if k in ['loss_func_1', 'loss_func_2', 'loss_func_3', 'adv_loss_func']:
                setattr(self, k, get_loss_functions(config[k]) )
            else:
                setattr(self, k, v)

    def check_cols(self):
        self.data = self.data.astype({col: np.float for col in self.continous_cols})

    def training_epoch_end(self, outs):
        if self.current_epoch == 0:
            self.logger.log_hyperparams(self.hparams)

    def transform(self, x, return_tensor=True):
        assert isinstance(x, pd.DataFrame)
        x_cont = self.normalizer.transform(x[self.continous_cols]) if self.continous_cols else np.array([[] for _ in range(len(x))])
        x_cat = self.encoder.transform(x[self.discret_cols]) if self.discret_cols else np.array([[] for _ in range(len(x))])
        x = np.concatenate((x_cont, x_cat), axis=1)
        return torch.from_numpy(x).float() if return_tensor else x

    def inverse_transform(self, x, return_tensor=True):
        """x should be a transformed tensor"""
        cat_idx = self.cat_idx
        # inverse transform
        x_cont_inv = self.normalizer.inverse_transform(x[:, :cat_idx].cpu())
        x_cat_inv = self.encoder.inverse_transform(x[:, cat_idx:].cpu()) if self.discret_cols else np.array([[] for _ in range(len(x))])
        x = np.concatenate((x_cont_inv, x_cat_inv), axis=1)
        return torch.from_numpy(x).float() if return_tensor else x

    def check_cont_robustness(self, x, c, c_y):
        cat_idx = self.cat_idx
        # inverse transform
        x_cont_inv = self.normalizer.inverse_transform(x[:, :cat_idx].cpu())
        c_cont_inv = self.normalizer.inverse_transform(c[:, :cat_idx].cpu())
        # calculate the diff between x and c
        cont_diff = np.abs(x_cont_inv - c_cont_inv) < self.threshold
        # total nums of differences
        total_diffs = np.sum(cont_diff.any(axis=1))
        # new continous cf
        c_cont_hat = np.where(cont_diff, x_cont_inv, c_cont_inv)
        c[:, :cat_idx] = torch.from_numpy(self.normalizer.transform(c_cont_hat))
        c_y_hat = self.predict(c)
        return ((c_y_hat > .5) != (c_y > .5)).sum(), total_diffs

    def cat_normalize(self, c, hard=False):
        # categorical feature starting index
        return cat_normalize(c, self.cat_arrays, self.cat_idx, hard=hard)

    def prepare_data(self):
        def split_x_and_y(data: pd.DataFrame):
            X = data[data.columns[:-1]]
            y = data[data.columns[-1]]
            return X, y

        X, y = split_x_and_y(self.data)

        # preprocessing
        if self.normalizer:
            X_cont = self.normalizer.transform(X[self.continous_cols])
        else:
            self.normalizer = MinMaxScaler()
            X_cont = self.normalizer.fit_transform(
                X[self.continous_cols]) if self.continous_cols else np.array([[] for _ in range(len(X))])

        if self.encoder:
            X_cat = self.encoder.transform(X[self.discret_cols])
        else:
            self.encoder = OneHotEncoder(sparse=False)
            X_cat = self.encoder.fit_transform(
                X[self.discret_cols]) if self.discret_cols else np.array([[] for _ in range(len(X))])
        X = np.concatenate((X_cont, X_cat), axis=1)
        pl_logger.info(f"x_cont: {X_cont.shape}, x_cat: {X_cat.shape}")

        self.cat_arrays = self.encoder.categories_ if self.discret_cols else []
        pl_logger.info(X.shape)
        assert X.shape[-1] == self.enc_dims[0], f'The input dimension X (shape: {X.shape[-1]})  != encoder_dims[0]: {self.enc_dims}'

        # prepare train & test
        train_X, test_X, train_y, test_y = train_test_split(X, y.to_numpy(), shuffle=False)
        if self.sample_frac:
            train_size = int(len(train_X) * self.sample_frac)
            train_X, train_y = train_X[:train_size], train_y[:train_size]
        self.train_dataset = NumpyDataset(train_X, train_y)
        self.val_dataset = NumpyDataset(test_X, test_y)
        self.test_dataset = self.val_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size,
                          pin_memory=True, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size * 4,
                          pin_memory=True, shuffle=False, num_workers=0)

    def test_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,
                          pin_memory=True, shuffle=False, num_workers=0)

## Baseline Model

In [ ]:
x = torch.tensor([0, 1, 1, 0])
x[x == 1]

tensor([1, 1])

In [ ]:
torch.where(x==1, torch.rand(4) * 0.15 + 0.8, torch.rand(4) * 0.15 + 0.05)

tensor([0.0800, 0.8633, 0.8413, 0.1227])

In [ ]:
# export utils
def uniform(shape: tuple, r1: float, r2: float, device=None, requires_grad=False):
    assert r1 < r2
    return (r2 - r1) * torch.rand(*shape, device=device, requires_grad=requires_grad) + r1

In [ ]:
# export
class PredictiveTrainingModule(BaseModule):
    def __init__(self, config: Dict):
        super().__init__(config)

    def prepare_lime_configs(self):
        features_name = self.continous_cols + self.discret_cols


    def model_forward(self, x):
        raise NotImplementedError

    def forward(self, *x):
        return self.model_forward(x)

    def predict(self, x):
        """x has not been preprocessed"""
        y_hat = self(x)
        return torch.round(y_hat)

    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=self.lr)

    def training_step(self, batch, batch_idx):
        # batch
        *x, y = batch
        # fwd
        y_hat = self(*x)
        # loss
        y = smooth_y(y)
        loss = F.binary_cross_entropy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('train/train_loss_1', loss, on_step=True, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        # batch
        *x, y = batch
        # fwd
        y_hat = self(*x)
        # loss
        loss = F.binary_cross_entropy(y_hat, y)
        score = accuracy(y_hat > .5, y.int())
        return {'score': score, 'val_loss': loss}

    def validation_epoch_end(self, val_outs):
        avg_loss = torch.stack([output['val_loss'] for output in val_outs]).mean()
        avg_score = torch.stack([output['score'] for output in val_outs]).mean()
        self.log('val/val_loss', avg_loss)
        self.log('val/pred_accuracy', avg_score)

## Counterfactual Model

### Helper Functions

In [ ]:
# export utils
def hinge_loss(input, target):
    """
    reference:
    - https://github.com/interpretml/DiCE/blob/a772c8d4fcd88d1cab7f2e02b0bcc045dc0e2eab/dice_ml/explainer_interfaces/dice_pytorch.py#L196-L202
    - https://en.wikipedia.org/wiki/Hinge_loss
    """
    input = torch.log((abs(input - 1e-6) / (1 - abs(input - 1e-6))))
    all_ones = torch.ones_like(target)
    target = 2 * target - all_ones
    loss = all_ones - torch.mul(target, input)
    loss = F.relu(loss)
    return torch.norm(loss)

In [ ]:
# x = torch.tensor([[0.6, 0.7, 0.1, 0.8]])
x = torch.rand(4, 1)
target = torch.tensor([[1, 1, 0, 1]])

print(f"x: {x} ")
print(f"hinge: {hinge_loss(x, target)} ")


x: tensor([[0.2904],
        [0.3001],
        [0.0508],
        [0.3926]]) 
hinge: 8.592495918273926 


### Counterfactual Model

In [ ]:
logits = torch.randn(1, 20)
logits[:, 2:] = F.gumbel_softmax(logits[:, 2:], hard=True)
logits

tensor([[ 0.9465, -0.0229,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000]])

In [ ]:
# export utils
def cat_normalize(c, cat_arrays, cat_idx, hard=False):
    # categorical feature starting index
    for col in cat_arrays:
        cat_end_idx = cat_idx + len(col)
        if hard:
            c[:, cat_idx: cat_end_idx] = F.gumbel_softmax(c[:, cat_idx: cat_end_idx].clone(), hard=hard)
        else:
            c[:, cat_idx: cat_end_idx] = F.softmax(c[:, cat_idx: cat_end_idx].clone(), dim=-1)
        cat_idx = cat_end_idx
    return c

def l1_mean(x, c):
    return F.l1_loss(x, c, reduction='mean') / x.abs().mean() # MAD

_loss_functions = {
    'cross_entropy': F.binary_cross_entropy,
    'l1': F.l1_loss,
    'l1_mean': l1_mean,
    'mse': F.mse_loss
}

def get_loss_functions(f_name: str):
    assert f_name in _loss_functions.keys(), f'function name "{f_name}" is not in the loss function list {_loss_functions.keys()}'
    return _loss_functions[f_name]

_optimizers = {
    'adam': torch.optim.Adam
}

def get_optimizers(o_name: str):
    assert o_name in _optimizers.keys(), f'optimizer name "{o_name}" is not in the optimizer list {_optimizers.keys()}'
    return _optimizers[o_name]

def smooth_y(y):
    return torch.where(y == 1,
                       uniform(y.size(), 0.8, 0.95, device=y.device),
                       uniform(y.size(), 0.05, 0.2, device=y.device))

def use_grad(*models, requires_grad: bool):
    for model in models:
        assert isinstance(model, nn.Module), f"{model} is not a `nn.Module` "
        for p in model.parameters():
            p.requires_grad = requires_grad

In [ ]:
# export

class CounterfactualTrainingModule(BaseModule, GlobalExplainerBase):
    def __init__(self, config: Dict):
        super().__init__(config)

    def forward(self, x: torch.Tensor, hard: bool = False) -> Tuple[torch.Tensor, torch.Tensor]:
        """return predicted probability and counterfactual explanation

        Args:
            x (torch.Tensor): input instance (processed by `self.encoder` and `self.normalizer`)
            hard (bool, optional): categorical features in cf is in one-hot-encoding or not. Defaults to False (in softmax format).

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: predicted probability and counterfactual explanation

        """
        y, cf = self.model_forward(x)
        cf = self.cat_normalize(cf, hard=hard)
        return y, cf

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """return predicted label

        Args:
            x (torch.Tensor): input instance (processed by `self.encoder` and `self.normalizer`)

        Returns:
            torch.Tensor: predicted label
        """
        y_hat, c = self.model_forward(x)
        return torch.round(y_hat)

    def generate_cf(self, x, clamp=False):
        self.freeze()
        y, c = self.model_forward(x)
        if clamp:
            c = torch.clamp(c, 0., 1.)
        return self.cat_normalize(c, hard=True)

    def _loss_functions(self, x, c, y, y_hat, y_prime=None, y_prime_mode='predicted', is_val=False):
        """
        x: input value
        c: conterfactual example
        y: ground truth
        y_hat: predicted result
        y_prime_mode: 'label' or 'predicted'
        """
        # flip zero/one
        if y_prime == None:
            if y_prime_mode == 'label':
                y_prime = torch.ones(y.shape) - y
            elif y_prime_mode == 'predicted':
                y_prime = (y_hat < .5).clone().detach().float()

        c_y, _ = self(c)
        # loss functions
        if self.smooth_y and not is_val:
            y = smooth_y(y)
            y_prime = smooth_y(y_prime)
        l_1 = self.loss_func_1(y_hat, y)
        l_2 = self.loss_func_2(x, c)
        l_3 = self.loss_func_3(c_y, y_prime)

        return l_1, l_2, l_3

    def _loss_compute(self, l_1, l_2, l_3):
        return self.lambda_1 * l_1 + self.lambda_2 * l_2 + self.lambda_3 * l_3

    def _logging_loss(self, l_1, l_2, l_3, stage: str, on_step: bool = False):
        self.log(f'{stage}/{stage}_loss_1', l_1, on_step=on_step, on_epoch=True, prog_bar=False, logger=True, sync_dist=True)
        self.log(f'{stage}/{stage}_loss_2', l_2, on_step=on_step, on_epoch=True, prog_bar=False, logger=True, sync_dist=True)
        self.log(f'{stage}/{stage}_loss_3', l_3, on_step=on_step, on_epoch=True, prog_bar=False, logger=True, sync_dist=True)

    def configure_optimizers(self):
        opt_1 = torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=self.lr)
        opt_2 = torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=self.lr)
        return (opt_1, opt_2)

    def predictor_loss(self, x, y):
        # fwd
        y_hat, c = self(x)
        y = smooth_y(y) if self.smooth_y else y
        return self.loss_func_1(y_hat, y)

    def predictor_step(self, l_1, opt):
        p_loss = self.lambda_1 * l_1
        self.manual_backward(p_loss, opt)
        nn.utils.clip_grad_norm_(self.parameters(), 0.5)
        opt.step()
        return p_loss

    def explainer_loss(self, x):
        y_hat, cf = self(x)
        cf_y, _ = self.model_forward(cf)
        y_prime =  (1. - torch.round(y_hat)).clone().detach()

        if self.smooth_y:
            y_prime = smooth_y(y_prime)
        l_2 = self.loss_func_2(x, cf)
        l_3 = self.loss_func_3(cf_y, y_prime)
        return l_2, l_3

    def explainer_step(self, l_2, l_3, opt):
        e_loss = self.lambda_2 * l_2 + self.lambda_3 * l_3
        self.manual_backward(e_loss, opt)
        nn.utils.clip_grad_norm_(self.parameters(), 0.5)
        opt.step()
        return e_loss

    def training_step(self, batch, batch_idx, optimizer_idx):
        # batch
        x, y = batch
        # fwd
        y_hat, c = self(x)
        # define optimizers
        opt_1, opt_2 = self.optimizers()

        ##########################
        # predictor optimization #
        ##########################
        l_1 = self.predictor_loss(x, y)
        p_loss = self.predictor_step(l_1, opt=opt_1)
        self.log('train/p_loss', p_loss, on_step=False, on_epoch=True, sync_dist=True)

        ##########################
        # explainer optimization #
        ##########################
        l_2, l_3 = self.explainer_loss(x)
        e_loss = self.explainer_step(l_2, l_3, opt=opt_2)
        self.log('train/e_loss', e_loss, on_step=False, on_epoch=True, sync_dist=True)

        # Logging to TensorBoard by default
        self._logging_loss(l_1, l_2, l_3, stage='train', on_step=False)

    def validation_step(self, batch, batch_idx):
        # batch
        x, y = batch
        # fwd
        y_hat, c = self(x, hard=True)
        c_y, _ = self(c)
        # loss
        l_1, l_2, l_3 = self._loss_functions(x, c, y, y_hat, is_val=True)
        loss = self.lambda_3 * l_3 + self.lambda_2 * l_2
        # logging val loss
        self._logging_loss(l_1, l_2, l_3, stage='val')

        # metrics
        cat_idx = len(self.continous_cols)

        pred_acc = accuracy(y_hat > .5, y.int())
        cf_proximity = torch.abs(x - c).sum(dim=1).mean()
        cf_acc = accuracy(c_y > .5, (y_hat < .5).int())

        # print counterfactual results
        # log = self._logging_cf_results(x, c, y, y_hat, c_y)
        log = None

        # logging robustness on manipulating small
        diffs, total_num = self.check_cont_robustness(x, c, c_y)
        sensitivity = 1 - diffs / total_num if total_num != 0 else 1.

        metrics = {
            'val/val_loss': loss, 'val/pred_accuracy': pred_acc,
            'val/cf_proximity': cf_proximity, 'val/sensitivity': sensitivity,
            'val/cf_accuracy': cf_acc
        }

        self.log_dict(metrics, on_step=False, on_epoch=True, sync_dist=True)


class AdvCounterfactualTrainingModule(CounterfactualTrainingModule, ABC):
    def __init__(self, config: Dict):
        super().__init__(config)
        self._DEFAULT_PARAMS.update({
            'epsilon': 0.1,
            'alpha': 'auto',
        })
        self._update_params(config)

    def _perturb_input(self, x: torch.Tensor, delta: torch.Tensor):
        raise NotImplementedError("_perturb_input(x, delta) is not implemented")

class AdvBiLevelCounterfactualTrainingModule(AdvCounterfactualTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        self._DEFAULT_PARAMS.update({
            'eps_scheduler': 'linear',
            'n_steps': 7,
            'k': 2,
            'adv_loss_func': "cross_entropy",
            'adv_val_models': None
        })
        self._update_params(config)
        self.glob_epsilon = self.epsilon
        self.epsilon = 0.

    def predictor_loss(self, x, y, target_model_name='org'):
        # fwd
        if target_model_name is 'org':
            y_hat, _ = self.model.model_forward(x)
        elif target_model_name is 'dup':
            y_hat, _ = self.model_dup.model_forward(x)
        else:
            raise ValueError(f"target should be one of ['org', 'dup'], but got `{target_model_name}`")
        y = smooth_y(y) if self.smooth_y else y
        return self.loss_func_1(y_hat, y)

    def _perturb_input(self, x: torch.Tensor, delta: torch.Tensor, model_dup: Optional[nn.Module] = None):
        if model_dup is None:
            model_dup = self.model_dup
        if torch.norm(delta) != 0:
            x = self.cat_normalize(x + delta)
        y_hat, cf = self.model.model_forward(x)
        cf = self.cat_normalize(cf, hard=False)
        cf_y, _ = model_dup.model_forward(cf)
        y_prime = (1. - torch.round(y_hat)).clone().detach()
        return y_hat, cf, cf_y, y_prime

    def explainer_loss(self, x: torch.Tensor, delta: Optional[torch.Tensor]=None, model_dup: Optional[nn.Module] = None):
        if delta is None:
            delta = torch.tensor([0.])
        y_hat, cf, cf_y, y_prime = self._perturb_input(x, delta, model_dup)
        self.log("train/robust_validity", accuracy(cf_y, y_prime.int()), on_step=True, on_epoch=True, sync_dist=True)

        if self.smooth_y:
            y_prime = smooth_y(y_prime)
        l_2 = self.loss_func_2(x, cf)
        l_3 = self.loss_func_3(cf_y, y_prime)
        return l_2, l_3

    def adv_loss(self, x: torch.Tensor, delta: Optional[torch.Tensor]=None, model_dup: Optional[nn.Module] = None):
        if delta is None:
            delta = torch.tensor([0.])
        y_hat, cf, cf_y, y_prime = self._perturb_input(x, delta, model_dup)
        adv_loss = self.adv_loss_func(cf_y, y_prime)
        return adv_loss

    def fast_adv_training(self, x, epsilon):
        alpha = epsilon * 1.25
        # init delta
        delta = torch.zeros_like(x).uniform_(-epsilon, epsilon)
        delta.requires_grad = True
        # explainer loss and calculate delta.grad
        l_2, l_3 = self.explainer_loss(x, delta)
        loss = self.lambda_2 * l_2 + self.lambda_3 * l_3
        loss.backward()
        # fast-sign gradient descent
        scaled_g = delta.grad.detach().sign()
        delta.data = l_inf_proj(delta + alpha * scaled_g, eps=epsilon, cat_idx=self.cat_idx)
        delta.grad.zero_()
        return delta.detach()

    def bilevel_adv_training(self, batch, _opt):
        x, y = batch
        # alpha = self.epsilon * 1.25
        alpha = self.epsilon * 2.5 / self.n_steps
        # init delta randomly
        delta = torch.zeros_like(x).uniform_(-self.epsilon, self.epsilon)
        delta.requires_grad = True

        use_grad(self, requires_grad=False)
        use_grad(self.model_dup.predictor, self.model_dup.pred_linear, requires_grad=True)
        opt = torch.optim.Adam(self.model_dup.parameters(), lr=self.lr)
        adv_loss = 0.
        val_loss = self.adv_loss(x, delta=None, model_dup=self.model)
        # _, val_loss = self.explainer_loss(x)

        for i in range(self.n_steps):
            with higher.innerloop_ctx(self.model_dup, opt, copy_initial_weights=True) as (fmodel, diffopt):
                for j in range(self.k):
                    _x = self.cat_normalize(x + delta)
                    y_hat, _ = fmodel(_x)

                    loss = self.loss_func_1(y_hat, y)
                    params = diffopt.step(loss)

                if i < self.n_steps - 1:
                    loss = self.adv_loss(x, delta=None, model_dup=fmodel)
                    loss.backward()

                    l = len(x.shape) - 1
                    g = delta.grad.detach()
                    g_norm = torch.norm(g.view(g.shape[0], -1), dim=1).view(-1, *([1]*l))
                    scaled_g = g / (g_norm + 1e-10)
                    delta.data = l2_proj(delta + alpha * scaled_g, eps=self.epsilon, cat_idx=self.cat_idx)

                    # scaled_g = delta.grad.detach().sign()
                    # delta.data = l_inf_proj(delta + alpha * scaled_g, eps=self.epsilon, cat_idx=self.cat_idx)
                    delta.grad.zero_()

                self.model_dup.load_state_dict(fmodel.state_dict())
                adv_loss = loss.detach()
        self.log('train/adv_loss', adv_loss, on_step=True)
        self.log('train/adv_loss_increment', adv_loss - val_loss, on_step=True)


    # def bilevel_adv_training(self, batch, opt):
    #     x, y = batch
    #     delta = torch.zeros_like(x)
    #     for i in range(self.n_steps):
    #         # STEP 1: adversarial training to invalidate CF generator
    #         use_grad(self, requires_grad=False)
    #         if self.epsilon == 0.:
    #             # only apply unrolling to the predictor
    #             delta = 0.
    #         else:
    #             delta = self.fast_adv_training(x, self.epsilon)

    #         # STEP 2: update predictor
    #         use_grad(self.model_dup.predictor, self.model_dup.pred_linear, requires_grad=True)
    #         for _ in range(self.k):
    #             x = self.cat_normalize(x + delta)
    #             p_loss = self.predictor_loss(x, y, 'dup')
    #             self.manual_backward(p_loss, opt)
    #             nn.utils.clip_grad_norm_(self.model_dup.parameters(), 0.5)
    #             opt.step()

    def schedule_eps(self):
        num_batches = len(self.train_dataloader())
        max_steps = self.trainer.max_epochs * num_batches

        self.epsilon = self.glob_epsilon * ((self.global_step + 1) / max_steps)
        self.log('train/eps', self.epsilon)

    def on_train_epoch_start(self) -> None:
        max_epochs = self.trainer.max_epochs
        current_epoch = self.current_epoch
        if self.eps_scheduler is None or self.eps_scheduler == "static":
            self.epsilon = self.glob_epsilon
        elif self.eps_scheduler == "linear":
            self.epsilon = self.glob_epsilon * ((current_epoch + 1) / max_epochs)
        elif self.eps_scheduler == "log":
            self.epsilon = self.glob_epsilon * (np.log(current_epoch + 1) / np.log(max_epochs))
        else:
            raise ValueError(f"config['eps_scheduler'] should be one of `[None, 'static', 'linear', 'log']`, but got `{self.eps_scheduler}`")

        self.log('train/eps', self.epsilon)
        # self.model_dup.load_state_dict(self.model.state_dict())


    def training_step(self, batch, batch_idx, optimizer_idx):
        x, y = batch
        # define optimizers
        opt_1, opt_2 = self.optimizers()

        ##########################
        # predictor optimization #
        ##########################
        use_grad(self, requires_grad=True)
        l_1 = self.predictor_loss(x, y, 'org')
        p_loss = self.predictor_step(l_1, opt=opt_1)
        self.log('train/p_loss', p_loss, on_step=False, on_epoch=True, sync_dist=True)

        # update predictive weights
        self.model_dup.load_state_dict(self.model.state_dict())

        ########################
        # adversarial training #
        ########################
        use_grad(self, requires_grad=False)
        self.bilevel_adv_training(batch, opt_1)

        ##########################
        # explainer optimization #
        ##########################
        use_grad(self.model_dup.predictor, self.model_dup.pred_linear, requires_grad=False)
        use_grad(self.model.explainer, requires_grad=True)

        l_2, l_3 = self.explainer_loss(x)
        e_loss = self.explainer_step(l_2, l_3, opt=opt_2)
        self.log('train/e_loss', e_loss, on_step=False, on_epoch=True, sync_dist=True)

        # update explainer weight
        # self.model.explainer.load_state_dict(self.model_dup.explainer.state_dict())

        # Logging to TensorBoard by default
        self._logging_loss(l_1, l_2, l_3, stage='train', on_step=False)

    def _adv_validity(self, cf, y_prime):
        def robust_validity(model, cf, y_prime):
            cf_y_2 = model.predict(cf)
            return accuracy(cf_y_2, y_prime.int())

        w_1_val_arr, w_all_val_arr = [], []
        for model in self.adv_val_models['w=1']:
            w_1_validity = robust_validity(model, cf, y_prime)
            w_1_val_arr.append(w_1_validity)
        for model in self.adv_val_models['w=all']:
            w_all_validity = robust_validity(model, cf, y_prime)
            w_all_val_arr.append(w_all_validity)
        return {
            'val/w=1_validity': np.average(w_1_val_arr),
            'val/w=all_validity': np.average(w_all_val_arr),
        }

## Models

In [ ]:
# export net
class LinearBlock(nn.Module):
    def __init__(self, input_dim, out_dim, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(input_dim, out_dim),
            # nn.BatchNorm1d(num_features=out_dim),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.block(x)


class MultilayerPerception(nn.Module):
    def __init__(self, dims=[3, 100, 10]):
        super().__init__()
        layers  = []
        num_blocks = len(dims)
        for i in range(1, num_blocks):
            layers += [
                LinearBlock(dims[i-1], dims[i])
            ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class BaselineModel(PredictiveTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        assert self.enc_dims[-1] == self.dec_dims[0]
        self.model = nn.Sequential(
            MultilayerPerception(self.enc_dims),
            MultilayerPerception(self.dec_dims),
            nn.Linear(self.dec_dims[-1], 1)
        )

    def model_forward(self, x):
        # x = ([],)
        x, = x
        y_hat = torch.sigmoid(self.model(x))
        return torch.squeeze(y_hat, -1)

In [ ]:
# export net

class ConvBlock(nn.Module):
    def __init__(self, input_dim, out_dim, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(input_dim, out_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(num_features=out_dim),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        return self.block(x)

class MultilayerConv(nn.Module):
    def __init__(self, dims=[3, 100, 10]):
        super().__init__()
        layers  = []
        num_blocks = len(dims)
        for i in range(1, num_blocks):
            layers += [
                ConvBlock(dims[i-1], dims[i])
            ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
# export net

class _CounterfactualModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        enc_dims = config['enc_dims'] if 'enc_dims' in config.keys() else []
        dec_dims = config['dec_dims'] if 'dec_dims' in config.keys() else []
        exp_dims = config['exp_dims'] if 'exp_dims' in config.keys() else []

        assert enc_dims[-1] == dec_dims[0], f"(enc_dims[-1]={enc_dims[-1]}) != (exp_dims[0]={dec_dims[0]})"
        assert enc_dims[-1] == exp_dims[0], f"(enc_dims[-1]={enc_dims[-1]}) != (exp_dims[0]={exp_dims[0]})"

        self.encoder_model = MultilayerPerception(enc_dims)
        # predictor
        self.predictor = MultilayerPerception(dec_dims)
        self.pred_linear = nn.Linear(dec_dims[-1], 1)
        # explainer
        exp_dims = [x for x in exp_dims]
        exp_dims[0] = exp_dims[0] + dec_dims[-1]

        self.explainer = nn.Sequential(
            MultilayerPerception(exp_dims),
            nn.Linear(exp_dims[-1], enc_dims[0])
        )

    def latent_vec(self, x):
        z = self.encoder_model(x)
        pred = self.predictor(z)
        return z, pred

    def latent_output(self, z, pred):
        # prediction
        y_hat = self.latent_y_hat_output(pred)
        # counterfactual example
        z_p = torch.cat((z, pred), -1)
        cf = self.explainer(z_p)
        return y_hat, cf

    def latent_y_hat_output(self, pred):
        y_hat = torch.sigmoid(self.pred_linear(pred))
        return torch.squeeze(y_hat, -1)

    def latent_cf_output(self, z_p):
        return self.explainer(z_p)

    def model_forward(self, x):
        z, pred = self.latent_vec(x)
        return self.latent_output(z, pred)

    def forward(self, x):
        # don't use this method unless the `forward` function is needed
        return self.model_forward(x)


class _CounterfactualFramework(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        enc_dims = config['enc_dims'] if 'enc_dims' in config.keys() else []
        dec_dims = config['dec_dims'] if 'dec_dims' in config.keys() else []
        exp_dims = config['exp_dims'] if 'exp_dims' in config.keys() else []

        assert enc_dims[-1] == dec_dims[0], f"(enc_dims[-1]={enc_dims[-1]}) != (exp_dims[0]={dec_dims[0]})"
        assert enc_dims[-1] == exp_dims[0], f"(enc_dims[-1]={enc_dims[-1]}) != (exp_dims[0]={exp_dims[0]})"

        # predictor
        self.predictor = MultilayerPerception(enc_dims + dec_dims[1:])
        self.pred_linear = nn.Linear(dec_dims[-1], 1)
        # explainer
        # exp_dims = [x for x in exp_dims]
        # exp_dims[0] = exp_dims[0] + dec_dims[-1]

        self.explainer = nn.Sequential(
            MultilayerPerception(enc_dims + exp_dims[1:]),
            nn.Linear(exp_dims[-1], enc_dims[0])
        )

    def latent_vec(self, x):
        pred = self.predictor(x)
        return x, pred

    def latent_output(self, x, pred):
        # prediction
        y_hat = self.latent_y_hat_output(pred)
        # counterfactual example
        cf = self.explainer(x)
        return y_hat, cf

    def latent_y_hat_output(self, pred):
        y_hat = torch.sigmoid(self.pred_linear(pred))
        return torch.squeeze(y_hat, -1)

    def latent_cf_output(self, z_p):
        return self.explainer(z_p)

    def model_forward(self, x):
        z, pred = self.latent_vec(x)
        return self.latent_output(z, pred)

    def forward(self, x):
        # don't use this method unless the `forward` function is needed
        return self.model_forward(x)



class CounterfactualModel(CounterfactualTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        self.model = _CounterfactualModel(config)

    def latent_vec(self, x):
        return self.model.latent_vec(x)

    def latent_output(self, z, pred):
        return self.model.latent_output(z, pred)

    def latent_y_hat_output(self, pred):
        return self.model.latent_y_hat_output(pred)

    def latent_cf_output(self, z_p):
        return self.model.latent_cf_output(z_p)

    def model_forward(self, x):
        return self.model.model_forward(x)

In [ ]:
# export net

class AdvCounterfactualModel(AdvBiLevelCounterfactualTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        self.model = _CounterfactualModel(config)
        self.model_dup = _CounterfactualModel(config)
        self.model_dup.load_state_dict(self.model.state_dict())

    def latent_vec(self, x):
        return self.model.latent_vec(x)

    def latent_output(self, z, pred):
        return self.model.latent_output(z, pred)

    def latent_y_hat_output(self, pred):
        return self.model.latent_y_hat_output(pred)

    def latent_cf_output(self, z_p):
        return self.model.latent_cf_output(z_p)

    def model_forward(self, x):
        return self.model.model_forward(x)

class AdvCounterfactualFramework(AdvBiLevelCounterfactualTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        self.model = _CounterfactualFramework(config)
        self.model_dup = _CounterfactualFramework(config)
        self.model_dup.load_state_dict(self.model.state_dict())

    def latent_vec(self, x):
        return self.model.latent_vec(x)

    def latent_output(self, z, pred):
        return self.model.latent_output(z, pred)

    def latent_y_hat_output(self, pred):
        return self.model.latent_y_hat_output(pred)

    def latent_cf_output(self, z_p):
        return self.model.latent_cf_output(z_p)

    def model_forward(self, x):
        return self.model.model_forward(x)

In [ ]:
# export net
class ConvCounterNet(CounterfactualTrainingModule):
    def __init__(self, config):
        super().__init__(config)
        assert self.enc_dims[-1] == self.dec_dims[0], f"(enc_dims[-1]={self.enc_dims[-1]}) != (exp_dims[0]={self.dec_dims[0]})"
        assert self.enc_dims[-1] == self.exp_dims[0], f"(enc_dims[-1]={self.enc_dims[-1]}) != (exp_dims[0]={self.enc_dims[0]})"

        self.encoder_model = MultilayerConv(self.enc_dims)
        # predictor
        self.predictor = MultilayerConv(self.dec_dims)
        self.pred_linear = nn.Linear(self.dec_dims[-1], 1)
        # explainer
        exp_dims = [x for x in self.exp_dims]
        exp_dims[0] = self.exp_dims[0] + self.dec_dims[-1]

        self.explainer = nn.Sequential(
            MultilayerPerception(exp_dims),
            nn.Linear(self.exp_dims[-1], self.enc_dims[0])
        )

    def model_forward(self, x):
        x = x.unsqueeze(dim=-1)
        x = self.encoder_model(x)
        # predicted y_hat
        pred = self.predictor(x)
        y_hat = torch.sigmoid(self.pred_linear(pred.squeeze(-1)))
        # counterfactual example
        x = torch.cat((x, pred), 1).squeeze(-1)
        c = self.explainer(x)
        return torch.squeeze(y_hat, -1), c